# RAG CHATBOT WITH CHAT MEMORY USING langchain- **create_history_aware_retriever**
- Import Library with OpeAPI keyand Goodle API Key
- Scrap content from webpage or html page using beautifulsoup
	- https://lilianweng.github.io/posts/2023-06-23-agent/
- Text split and chunking
- Apply embedding on chunks
- Read rag prompt template from Langchain hub
	- https://smith.langchain.com/hub/rlm/rag-prompt
- Create RAG chain
- ADD Chat MEMORY using create_history_aware_retriever
	- This Create a chain that takes conversation history and returns documents.

	- If there is no chat_history, then the input is just passed directly to the retriever. If there is chat_history, then the prompt and LLM will be used to generate a search query. That search query is then passed to the retriever.
- Prompt to rewrite the Q based on chat history
- Enable create_history_aware_retriever
- QA Prompt Template
- Create chain for qa prompt using create_stuff_documents_chain
- Create Final chain combaining history_aware_retriever and question_answer_chain
- Test the sequence of Q - 1st Q
- Append History
- Test the sequence of Q - 2nd Q

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4 -q
!pip install -U langchain_google_genai -q
!pip install sentence_transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.5 MB/s eta 0:00:00
 

## Import Library with OpeAPI keyand Goodle API Key

In [ ]:
import os
from google.colab import userdata
# OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
# os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI

#Invoke Google gemini-1.5-pro
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
llm.invoke("what is a value of 2+3?").content

'2 + 3 = 5\n'

In [5]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
import bs4  #beautiful soup

## Scrap content from webpage or html page using beautifulsoup
- https://lilianweng.github.io/posts/2023-06-23-agent/

In [8]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs=loader.load()
print(docs[0].page_content[:1000])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng


Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.
Agent System Overview#
In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:

Planning

Subgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.
Reflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.


Memory

Short-term memory: I 

## Text split and chunking

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits =text_splitter.split_documents(docs)
splits[0]

Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refin

## Apply embedding on chunks

In [11]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en-v1.5"
encode_kwargs = {'normalize_embeddings': True}

embedding_function=HuggingFaceBgeEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
#Apply embedding and save in chroma db
vectorstore = Chroma.from_documents(splits,embedding_function)
retriever=vectorstore.as_retriever()  #Retriver

In [12]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-pro', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7f219813b190>, default_metadata=())

## Read rag prompt template from Langchain hub
- https://smith.langchain.com/hub/rlm/rag-prompt

In [15]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

"You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:")

In [19]:
#This is to concat retrived info
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

## Create RAG chain

In [20]:
rag_chain=(
    {"context" : retriever | format_docs, "question": RunnablePassthrough()}  #Q get from runtime and pass to retriver, retrived infp passed format_docs to join it
    | prompt   #Then that ['context', 'question'] passed to prompt
    | llm
    | StrOutputParser()
)

## Output from RAG Chain

In [21]:
rag_chain.invoke("can you give me a Types of Memory?")

'Sensory memory stores sensory information briefly. Short-term or working memory holds current information for immediate tasks. Long-term memory stores information for extended periods, including explicit (consciously recalled) and implicit (unconscious, skill-based) memories.\n'

# ADD Chat MEMORY using create_history_aware_retriever

- This Create a chain that takes **conversation history and returns documents**.

- If there is no chat_history, then the input is just passed directly to the retriever. If there is chat_history, then the prompt and LLM will be used to generate a **search query**. That search query is then passed to the retriever.

In [22]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

## Prompt to rewrite the Q based on chat history

In [23]:
# Prompt to rewrite the Q based on chat history
contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

In [24]:
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

## Enable create_history_aware_retriever

In [26]:
history_aware_retriever = create_history_aware_retriever(
    llm,  # llm used
    retriever,  #Retriver
    contextualize_q_prompt,  #contextualize_q_prompt which gives reformed Q based on history and Q
)

history_aware_retriever

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7f2089f06d70>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessage

In [27]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

## QA Prompt Template

In [28]:
qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""


qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


## Create chain for qa pr0mpt using create_stuff_documents_chain

In [29]:
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)


## Create Final chain combaining history_aware_retriever and question_answer_chain

In [30]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [34]:
from langchain_core.messages import HumanMessage

chat_history=[]

## Test the sequence of Q - 1st Q

In [35]:
question1= "What is Task Decomposition?"
response1=rag_chain.invoke({"input":question1,"chat_history": chat_history})
print(response1['answer'])

Task decomposition is a planning technique used in LLMs to break down complex tasks into smaller, more manageable steps.  Chain-of-thought (CoT) prompting is a standard method for task decomposition, enhancing model performance by guiding the LLM to "think step by step". Tree of Thoughts extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of thoughts.



Task Decomposition is the process of breaking down a complex task into smaller, more manageable subtasks. This approach, often used in Chain-of-Thought prompting, helps LLMs understand and address complex problems by tackling them step-by-step. The decomposition can be facilitated by the LLM itself, guided by specific instructions, or provided by human input.

In [37]:
response1

{'input': 'What is Task Decomposition?',
 'chat_history': [HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}),
  'Task decomposition is a planning technique used in LLMs to break down complex tasks into smaller, more manageable steps.  Chain-of-thought (CoT) prompting is a standard method for task decomposition, enhancing model performance by guiding the LLM to "think step by step". Tree of Thoughts extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of thoughts.\n'],
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with e

## Append History

In [36]:
chat_history.extend([HumanMessage(content=question1), response1["answer"]])

## Test the sequence of Q - 2nd Q

In [39]:
question2="what are common ways of doing it?"
response2=rag_chain.invoke({"input":question2,"chat_history": chat_history})
print(response2['answer'])

Task decomposition can be achieved by prompting the LLM with simple instructions to list steps or subgoals.  More specific instructions tailored to the task, such as "Write a story outline," can also be used.  Finally, humans can directly provide the task decomposition.



Task decomposition can be achieved by prompting the LLM with simple instructions to list steps or subgoals.  More specific instructions tailored to the task, such as "Write a story outline," can also be used.  Finally, humans can directly provide the task decomposition.


In [41]:
response2

{'input': 'what are common ways of doing it?',
 'chat_history': [HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}),
  'Task decomposition is a planning technique used in LLMs to break down complex tasks into smaller, more manageable steps.  Chain-of-thought (CoT) prompting is a standard method for task decomposition, enhancing model performance by guiding the LLM to "think step by step". Tree of Thoughts extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of thoughts.\n'],
 'context': [Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Finite context length: The restricted context capacity limits the inclusion of historical information, detailed instructions, API call context, and responses. The design of the system has to work with this limited communication bandwidth, while mechanisms like self-reflection to learn from past mistakes would benefi

## Append History

In [42]:
chat_history.extend([HumanMessage(content=question2), response2["answer"]])

## Test the sequence of Q - 3rd Q

In [43]:
question3="for writing a novel what i can do?"
response3=rag_chain.invoke({"input":question3,"chat_history": chat_history})
print(response3['answer'])

For writing a novel with task decomposition, you can prompt the LLM with instructions like "Write a story outline," or provide your own outline breaking the novel into chapters or sections.  You can also provide more detailed instructions, such as character sketches, plot points, or setting descriptions, for each part.



For writing a novel with task decomposition, you can prompt the LLM with instructions like "Write a story outline," or provide your own outline breaking the novel into chapters or sections.  You can also provide more detailed instructions, such as character sketches, plot points, or setting descriptions, for each part.

In [45]:
response3

{'input': 'for writing a novel what i can do?',
 'chat_history': [HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}),
  'Task decomposition is a planning technique used in LLMs to break down complex tasks into smaller, more manageable steps.  Chain-of-thought (CoT) prompting is a standard method for task decomposition, enhancing model performance by guiding the LLM to "think step by step". Tree of Thoughts extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure of thoughts.\n',
  HumanMessage(content='what are common ways of doing it?', additional_kwargs={}, response_metadata={}),
  'Task decomposition can be achieved by prompting the LLM with simple instructions to list steps or subgoals.  More specific instructions tailored to the task, such as "Write a story outline," can also be used.  Finally, humans can directly provide the task decomposition.\n'],
 'context': [Document(metadata={'source': 'htt

## END